In [1]:
import os

os.environ["CONFIG_APP_DIR"] = "/app/config"


In [2]:
import sys
import os
import pickle
os.chdir("/app")
from omegaconf import OmegaConf

import json
from etl_pipeline.data_processor_engine.json_engine.json_engine import JsonProcessingEngine
from etl_pipeline.custom.ms.payload_loader import PayloadLoader

/env/ds/anaconda/envs/pipeline/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
from etl_pipeline.config import pipeline_config
cn = pipeline_config.cn

In [4]:
# alert = load_alert()

In [5]:
# payload = load_alert()


# Definition

In [6]:
from pipelines.ms.ms_pipeline import MSPipeline
from etl_pipeline.config import pipeline_config


In [7]:
with open(f'notebooks/sample/wm_party_in_payload_format.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)



payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]


In [8]:
payload = payload_json

In [9]:
from etl_pipeline.config import load_agent_configs, pipeline_config

In [10]:
pipeline_config.reload_pipeline_config()
_ = load_agent_configs()

2022-04-13 07:58:00,862 — main — WARNING — load_agent_config:68 — Field in agent configuration for: ALL_CONCAT_NAMES is not registered field in pipeline.yaml
2022-04-13 07:58:00,867 — main — WARNING — load_agent_config:68 — Field in agent configuration for: ALL_PARTY1_NAME_FULL is not registered field in pipeline.yaml
2022-04-13 07:58:00,871 — main — WARNING — load_agent_config:68 — Field in agent configuration for: ALL_CONCAT_NAMES is not registered field in pipeline.yaml
2022-04-13 07:58:00,878 — main — WARNING — load_agent_config:68 — Field in agent configuration for: ALL_PARTY1_NAME_FULL is not registered field in pipeline.yaml
2022-04-13 07:58:00,883 — main — WARNING — load_agent_config:68 — Field in agent configuration for: ALL_PARTY1_ORGANIZATION_NAME is not registered field in pipeline.yaml
2022-04-13 07:58:00,992 — main — WARNING — load_agent_config:68 — Field in agent configuration for: AP_PARTY_TYPE is not registered field in pipeline.yaml


In [11]:
engine = JsonProcessingEngine(pipeline_config)
pipeline = MSPipeline(engine, config=pipeline_config)

2022-04-13 07:58:01,285 — main — WARNING — load_agent_config:68 — Field in agent configuration for: ALL_CONCAT_NAMES is not registered field in pipeline.yaml
2022-04-13 07:58:01,290 — main — WARNING — load_agent_config:68 — Field in agent configuration for: ALL_PARTY1_NAME_FULL is not registered field in pipeline.yaml
2022-04-13 07:58:01,295 — main — WARNING — load_agent_config:68 — Field in agent configuration for: ALL_CONCAT_NAMES is not registered field in pipeline.yaml
2022-04-13 07:58:01,299 — main — WARNING — load_agent_config:68 — Field in agent configuration for: ALL_PARTY1_NAME_FULL is not registered field in pipeline.yaml
2022-04-13 07:58:01,305 — main — WARNING — load_agent_config:68 — Field in agent configuration for: ALL_PARTY1_ORGANIZATION_NAME is not registered field in pipeline.yaml
2022-04-13 07:58:01,405 — main — WARNING — load_agent_config:68 — Field in agent configuration for: AP_PARTY_TYPE is not registered field in pipeline.yaml


## transform standardized to cleansed

In [12]:
payload = pipeline.transform_standardized_to_cleansed(payload)

2022-04-13 07:58:01,727 — main.etl_pipeline — WARNING — get_parties:168 — No parties
2022-04-13 07:58:01,728 — main.etl_pipeline — WARNING — get_accounts:178 — No accounts
2022-04-13 07:58:01,730 — main.etl_pipeline — WARNING — get_parties:168 — No parties
2022-04-13 07:58:01,731 — main.etl_pipeline — WARNING — get_accounts:178 — No accounts
2022-04-13 07:58:01,733 — main.etl_pipeline — WARNING — get_parties:168 — No parties
2022-04-13 07:58:01,734 — main.etl_pipeline — WARNING — get_accounts:178 — No accounts


ALL_CONNECTED_ACCOUNT_NAMES# Transform standardized to application

In [13]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

2022-04-13 07:58:02,214 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload named: hit_type_agent_country. Ignore for HIT TYPE AGENT
2022-04-13 07:58:02,215 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload named: hit_type_agent_job. Ignore for HIT TYPE AGENT
2022-04-13 07:58:02,216 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload named: hit_type_agent_other. Ignore for HIT TYPE AGENT
2022-04-13 07:58:02,216 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload named: hit_type_agent_country. Ignore for HIT TYPE AGENT
2022-04-13 07:58:02,216 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload named: hit_type_agent_job. Ignore for HIT TYPE AGENT
2022-04-13 07:58:02,217 — main.engine — WARNING — merge_to_target_col_from_source_col

### payload

In [14]:
len(new_payloads)

2

In [15]:
import pandas as pd
out_payload = pd.DataFrame([match for payload in new_payloads   for match in payload['watchlistParty']['matchRecords']])

out_payload['AP_TRIGGERS'].values[0]

{'Doe': {'ADDRESS1_LINE2': ['Jane Doe AA BBB'],
  'CONCAT_ADDRESS': ['Joe Black & Jane Doe 111 A 11TH ST AAA 1A New Jersey NJ']}}

In [16]:
out_payload[[i for i in out_payload.columns if "entity" in i]]

,entity,entityId,entityTextType,entityType,entityVersion,is_entity_match_agent_ap,is_entity_match_agent_wl,ap_all_is_entity_matchs_aggregated,wl_all_is_entity_matchs_aggregated
0,"{'addresses': [{'address1': '111 STREET 999', ...",908043,UPID,03,20150505194929,C,I,[C],[I]
1,{'addresses': {'address': {'address1': '111 ST...,908044,UPID,03,20150505194923,C,I,[C],[I]


In [17]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        
    for num, match in enumerate(payload['watchlistParty']['matchRecords']):
        assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        print(num, 'match')
        print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
        try:
            print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
        except:
            break


PAYLOAD
122438658
0 match
[] ['MAY 6, 1981']
[] []
PAYLOAD
122438659
0 match
[] ['MAY 6, 1981']
[] ['Poland', 'PL']


In [18]:
import pickle
with open("tests/shared/parsed_payload.pkl", "wb") as f:
    pickle.dump(new_payloads, f)

In [19]:
with open(f'notebooks/sample/wm_address_in_payload_format_2_input_3_match_records.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)



payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]


In [ ]:
payload = payload_json

In [ ]:
payload = pipeline.transform_standardized_to_cleansed(payload)

In [ ]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

In [ ]:
import pandas as pd
out_payload = pd.DataFrame([match for payload in new_payloads   for match in payload['watchlistParty']['matchRecords']])

out_payload[[i for i in out_payload.columns  if i.endswith("_aggregated")]]

In [ ]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        
    for num, match in enumerate(payload['watchlistParty']['matchRecords']):
        assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        print(num, 'match')
        print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
        try:
            print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
        except:
            break


In [ ]:
import pickle
with open("tests/shared/parsed_payload_2_payload.pkl", "wb") as f:
    pickle.dump(new_payloads, f)

In [ ]:
new_dict = {}
def return_key(dict_, prefix):
    
    if isinstance(dict_, list):
        for num, i in enumerate(dict_):
            return_key(i, prefix +f"[{num}]")
        return
    if isinstance(dict_, str) or dict_ is None:
        new_dict[prefix] = dict_
        return

    for key in dict_:
        if prefix:
            basic_prefix = prefix + "." + key
        else:
            basic_prefix = key
        return_key(dict_[key], basic_prefix)

In [33]:
with open(f'notebooks/sample/wm_party_payload_without_supplemental_info.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)



payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]


In [34]:
payload = payload_json

In [35]:
payload = pipeline.transform_standardized_to_cleansed(payload)

2022-04-13 08:06:10,408 — main.etl_pipeline — WARNING — get_parties:168 — No parties
2022-04-13 08:06:10,412 — main.etl_pipeline — WARNING — get_accounts:178 — No accounts
2022-04-13 08:06:10,419 — main.etl_pipeline — WARNING — get_parties:168 — No parties
2022-04-13 08:06:10,419 — main.etl_pipeline — WARNING — get_accounts:178 — No accounts
2022-04-13 08:06:10,421 — main.etl_pipeline — WARNING — get_parties:168 — No parties
2022-04-13 08:06:10,422 — main.etl_pipeline — WARNING — get_accounts:178 — No accounts


In [36]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

2022-04-13 08:06:10,624 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload named: hit_type_agent_country. Ignore for HIT TYPE AGENT
2022-04-13 08:06:10,624 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload named: hit_type_agent_job. Ignore for HIT TYPE AGENT
2022-04-13 08:06:10,625 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload named: hit_type_agent_other. Ignore for HIT TYPE AGENT
2022-04-13 08:06:10,625 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload named: hit_type_agent_country. Ignore for HIT TYPE AGENT
2022-04-13 08:06:10,626 — main.engine — WARNING — merge_to_target_col_from_source_cols_sql_expression:235 — No field in payload named: hit_type_agent_job. Ignore for HIT TYPE AGENT
2022-04-13 08:06:10,626 — main.engine — WARNING — merge_to_target_col_from_source_col

In [37]:
import pandas as pd
out_payload = pd.DataFrame([match for payload in new_payloads   for match in payload['watchlistParty']['matchRecords']])

out_payload[[i for i in out_payload.columns  if i.endswith("_aggregated")]]

,ap_all_dobs_aggregated,wl_all_dobs_aggregated,ap_all_documents_aggregated,wl_all_documents_aggregated,ap_all_company_names_aggregated,wl_all_company_names_aggregated,ap_all_names_aggregated,wl_all_names_aggregated,ap_all_employer_names_aggregated,wl_all_employer_names_aggregated,...,wl_all_hit_types_aggregated,ap_all_matched_tokens_tp_markeds_aggregated,wl_all_matched_tokens_tp_markeds_aggregated,ap_all_ap_name_tp_markeds_aggregated,wl_all_ap_name_tp_markeds_aggregated,ap_all_ap_id_tp_markeds_aggregated,wl_all_ap_id_tp_markeds_aggregated,ap_all_is_entity_matchs_aggregated,wl_all_is_entity_matchs_aggregated,all_hit_type_aggregated
0,[],"[MAY 6, 1981]",[],[],[],[Joe Ding],[],[Joe Ding],[],[Joe Ding],...,[],[],[],[],[],[],[],[C],[I],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."
1,[],"[MAY 6, 1981]",[],[],[],[Joe Dong],[],[Joe Dong],[],[Joe Dong],...,[],[],[],[],[],[],[],[C],[I],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],..."


In [38]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        
    for num, match in enumerate(payload['watchlistParty']['matchRecords']):
        assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        print(num, 'match')
        print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
        try:
            print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
        except:
            break


PAYLOAD
122438658
0 match
[] ['MAY 6, 1981']
[] []
PAYLOAD
122438659
0 match
[] ['MAY 6, 1981']
[] ['Poland', 'PL']


In [39]:
import pickle
with open("tests/shared/empty_payload.pkl", "wb") as f:
    pickle.dump(new_payloads, f)